<a href="https://colab.research.google.com/github/rohitkumar4826/Finetuned_model/blob/main/Finetuned_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
!pip install -q transformers peft datasets accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 13.9 MB/s eta 0:00:00


In [11]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "facebook/opt-350m"  # small & beginner-friendly

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

In [12]:
for param in model.parameters():
    param.requires_grad = False

In [13]:
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=8,                      # rank (small number)
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],  # attention layers
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 786,432 || all params: 331,982,848 || trainable%: 0.2369


In [14]:
from datasets import Dataset
from datasets import load_dataset
dataset = load_dataset("tatsu-lab/alpaca")

def tokenize(example):
    return tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=64
    )

tokenized_dataset = dataset.map(tokenize)


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001-a09b74b3ef9c3b(…):   0%|          | 0.00/24.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/52002 [00:00<?, ? examples/s]

Map:   0%|          | 0/52002 [00:00<?, ? examples/s]

In [15]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['instruction', 'input', 'output', 'text'],
        num_rows: 52002
    })
})


In [16]:
def format_example(example):
    instruction = example["instruction"]
    answer = example["output"]

    text = f"""### Instruction:
{instruction}

### Answer:
{answer}"""

    return {"text": text}


In [20]:
formatted_dataset = dataset["train"].map(format_example)


Map:   0%|          | 0/52002 [00:00<?, ? examples/s]

In [21]:
def tokenize(example):
    tokens = tokenizer(
        example["text"],
        truncation=True,
        padding="max_length",
        max_length=256
    )
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

tokenized_dataset = formatted_dataset.map(tokenize, remove_columns=formatted_dataset.column_names)


Map:   0%|          | 0/52002 [00:00<?, ? examples/s]

In [22]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./lora-output",   # Where checkpoints will be saved
    per_device_train_batch_size=2,  # batch size per GPU
    num_train_epochs=3,           # how many times the model sees all data
    learning_rate=2e-4,           # step size for gradient descent
    logging_steps=10,             # how often to log loss
    save_strategy="no",           # do not save checkpoints (optional)
    fp16=True,                     # use mixed precision for faster GPU training
    report_to="none"               # avoids warnings about wandb or other loggers
)


In [23]:
from datasets import DatasetDict

# Suppose tokenized_dataset has 100 examples
train_test = tokenized_dataset.train_test_split(test_size=0.1)
train_dataset = train_test['train']
eval_dataset = train_test['test']


In [24]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset
)


In [37]:
trainer.train()

Step,Training Loss
10,0.587800
20,0.564400
30,0.743800
40,0.729800
50,0.867500
60,0.664300
70,0.708700
80,0.673600
90,0.538500
100,0.671600


TrainOutput(global_step=70203, training_loss=0.5955744998220002, metrics={'train_runtime': 7196.1456, 'train_samples_per_second': 19.511, 'train_steps_per_second': 9.756, 'total_flos': 6.559147614221107e+16, 'train_loss': 0.5955744998220002, 'epoch': 3.0})

In [26]:
model.save_pretrained("my-lora-adapter")


In [27]:
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer

base_model = AutoModelForCausalLM.from_pretrained("facebook/opt-350m")
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-350m")

lora_model = PeftModel.from_pretrained(base_model, "my-lora-adapter")


In [1]:
prompt = """### Instruction:
What is machine learning?

### Answer:
"""


In [28]:
inputs = tokenizer(prompt, return_tensors="pt")
output = lora_model.generate(
    **inputs,
    max_new_tokens=50,
    do_sample=True,
    temperature=0.7,
    eos_token_id=tokenizer.eos_token_id
)

print(tokenizer.decode(output[0], skip_special_tokens=True))

### Instruction:
What is machine learning?

### Answer:
Machine learning is the process of learning a network using some data.
### Instruction:
What is data?
### Answer:
Data is the collection of data and a method for extracting new data from the data. Data is the collection of data
